In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df = pd.read_csv("/kaggle/input/mobile-price-classification/train.csv")

df.head()

df.describe()

By looking at the count of each column its evident that there are no missing values.

In [ ]:
df.head()

In [ ]:
df.shape

# Data Analysis

We will create a correlation plot in order to understand the correlation between the features.

In [ ]:
plt.figure(figsize = (17,10))
corrmat = df.corr()
sns.heatmap(corrmat, square = True)
plt.title('Correlation', fontsize = 20)
plt.show()

Our Price Range has strong correlation with RAM.

Also

* 3G and 4G
* pc(Primary Camera mega pixels) and fc(Front Camera mega pixels)
* px_weight(Pixel Resolution Width) and px_height(Pixel Resolution Height)
* sc_w(Screen Width of mobile in cm) and sc_h(Screen Height of mobile in cm)

features have highly positive correlation. For example as long as sc_w (screen width of mobile) increase, sc_h(screen height of mobile) is increasing.

In [ ]:
#price range correlation
corrmat.sort_values(by=["price_range"],ascending=False).iloc[0].sort_values(ascending=False)

# Data Visualization

In [ ]:
sns.countplot(x = "price_range", data = df)
plt.show()

In [ ]:
g = sns.FacetGrid(df, col="price_range", hue="price_range", palette="Set1",height=5
                   )
g = (g.map(sns.distplot, "ram").add_legend())

This is a positive co - relationship.

There are 4 types of price range:

* Type 1(low cost): RAM ranges between 0 to 2000 megabytes
* Type 2(medium cost):  RAM ranges between 0 to 3000 megabytes
* Type 3(high cost): RAM ranges between 1000 to 4000 megabytes
* Type 4(very high cost): RAM ranges between 2000 to 4000 megabytes

### **Battery Power - RAM and Price Range**

In [ ]:
g = sns.FacetGrid(df,hue = 'price_range', height = 10)
g.map(plt.scatter, 'ram','battery_power',alpha = 0.6)
g.add_legend()


### **3G - RAM and Price Range**

In [ ]:
plt.figure(figsize = (15,10))
sns.boxplot(x="three_g", y="ram", hue = "price_range", data = df)
plt.xlabel("3-G", fontsize = 20)
plt.ylabel("RAM", fontsize = 20)

plt.show()

0 - Not 3G

1- Has 3G

### **4G - RAM and Price Range**

In [ ]:
plt.figure(figsize = (15,10))
sns.boxplot(x="four_g", y="ram", hue = "price_range", data = df, palette = "gnuplot")
plt.xlabel("4-G", fontsize = 20)
plt.ylabel("RAM", fontsize = 20)

plt.show()

0 - Not 4G

1- Has 4G

### **Internal Memory - RAM and Price Range**

In [ ]:
g = sns.FacetGrid(df,hue = 'price_range', height = 10)
g.map(plt.scatter, 'ram','int_memory',alpha = 0.6, s = 400)
plt.xlabel("RAM", fontsize = 20)
plt.ylabel("Internal Memory", fontsize = 20)

g.add_legend()
plt.show()


Internal Memory has a distinct relationship with RAM and Price Range.

### **Dual SIM - RAM and Price Range**

In [ ]:
g = sns.FacetGrid(df, col="dual_sim", hue="price_range", palette="Set1",height=5
                   )
g = (g.map(sns.distplot, "ram").add_legend())

### **Face Camera Pixels - RAM and Price Range**

In [ ]:
g = sns.FacetGrid(df, hue = "price_range", height = 5)
g = (g.map(plt.scatter, "fc","ram").add_legend())

In [ ]:
g = sns.FacetGrid(df, hue = "price_range", height = 5)
g = (g.map(plt.scatter, "fc","ram").add_legend())

# Prediction Models

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
X = df[['battery_power', 'blue', 'clock_speed', 'dual_sim', 'fc', 'four_g',
       'int_memory', 'm_dep', 'mobile_wt', 'n_cores', 'pc', 'px_height',
       'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time', 'three_g',
       'touch_screen', 'wifi']]
y = df['price_range']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42, stratify = y)

## Random Forest Classification

### Random Forest Classification without feature engineering

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 70, criterion = 'entropy', random_state = 42)
classifier.fit(X_train, y_train)
classifier.score(X_test,y_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 70, criterion = 'gini', random_state = 42)
classifier.fit(X_train, y_train)
classifier.score(X_test,y_test)

'gini' criterion gives better accuracy of **87.83%** by a small margin.

### Random Forest Classification with feature engineering

In [ ]:
#feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_fc = sc.fit_transform(X_train)
X_test_fc = sc.transform(X_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier_fe = RandomForestClassifier(n_estimators = 70, criterion = 'gini', random_state = 42)
classifier_fe.fit(X_train_fc, y_train)
classifier_fe.score(X_test_fc,y_test)

In [ ]:
#making confusion matrix
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier_fe.predict(X_test_fc)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

Even after feature scaling the result is **same 87.83%**, now let's try dropping unnecessary columns.

In [ ]:
feature_list = list(df.columns)

In [ ]:
# Get numerical feature importances
importances = list(classifier_fe.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances]

In [ ]:
# list of x locations for plotting
x_values = list(range(len(importances)))
# Make a bar chart
plt.bar(x_values, importances, orientation = 'vertical', color = 'r', edgecolor = 'k', linewidth = 1.2)
# Tick labels for x axis
plt.xticks(x_values, feature_list, rotation='vertical')
# Axis labels and title
plt.ylabel('Importance'); plt.xlabel('Variable'); plt.title('Variable Importances');

In [ ]:
# List of features sorted from most to least important
sorted_importances = [importance[1] for importance in feature_importances]
sorted_features = [importance[0] for importance in feature_importances]
# Cumulative importances
cumulative_importances = np.cumsum(sorted_importances)
# Make a line graph
plt.plot(x_values, cumulative_importances, 'g-')
# Draw line at 95% of importance retained
plt.hlines(y = 0.95, xmin=0, xmax=len(sorted_importances), color = 'r', linestyles = 'dashed')
# Format x ticks and labels
plt.xticks(x_values, sorted_features, rotation = 'vertical')
# Axis labels and title
plt.xlabel('Variable'); plt.ylabel('Cumulative Importance'); plt.title('Cumulative Importances');

In [ ]:
# Find number of features for cumulative importance of 95%
# Add 1 because Python is zero-indexed
print('Number of features for 95% importance:', np.where(cumulative_importances > 0.95)[0][0] + 1)

Now we will drop the least important features.

In [ ]:
X_modified = df[['battery_power', 'clock_speed', 'fc',
       'int_memory', 'm_dep', 'mobile_wt', 'n_cores', 'pc', 'px_height',
       'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time']]

Training and evaluating on important features.

In [ ]:
from sklearn.model_selection import train_test_split
X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(X_modified, y, test_size = 0.3, random_state = 42, stratify = y)

In [ ]:
#feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_m = sc.fit_transform(X_train_m)
X_test_m = sc.transform(X_test_m)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier_m = RandomForestClassifier(n_estimators = 80, criterion = 'gini', random_state = 42)
classifier_m.fit(X_train_m, y_train)
classifier_m.score(X_test_m,y_test)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier_m.predict(X_test_m)
cm_m = confusion_matrix(y_test, y_pred)
print(cm_m)
accuracy_score(y_test, y_pred)

There was a slight increase in accuracy after feature engineering, **from 87.83% to 88.50%**.

In [ ]:
sns.heatmap(cm_m, annot = True,fmt='', cmap='Blues')

In [ ]:
# # Performance metrics
# errors = abs(y_pred - y_test)
# print('Average absolute error:', round(np.mean(errors), 2), 'degrees.')
# # Calculate mean absolute percentage error (MAPE)
# mape = 100 * (errors / y_test)
# # Calculate and display accuracy
# accuracy = 100 - np.mean(mape)
# print('Accuracy:', round(accuracy, 2), '%.')

## KNN Classification

### KNN classification without feature engineering.

In [ ]:
X = df[['battery_power', 'blue', 'clock_speed', 'dual_sim', 'fc', 'four_g',
       'int_memory', 'm_dep', 'mobile_wt', 'n_cores', 'pc', 'px_height',
       'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time', 'three_g',
       'touch_screen', 'wifi']]
y = df['price_range']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42, stratify = y)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier.fit(X_train, y_train)
classifier.score(X_test,y_test)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(X_test)
cm_knn = confusion_matrix(y_test, y_pred)
print(cm_knn)
accuracy_score(y_test, y_pred)

Wow! We already have an accuracy of 93.33% without feature engineering which is better than Random Forest Classification. Now let's see with feature engineering.

In [ ]:
sns.heatmap(cm_knn, annot = True,fmt='', cmap='Blues')

### KNN classification with feature engineering.

In [ ]:
#feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_fc = sc.fit_transform(X_train)
X_test_fc = sc.transform(X_test)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier_fc = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier_fc.fit(X_train_fc, y_train)
classifier_fc.score(X_test_fc,y_test)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier_fc.predict(X_test_fc)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

Okay so, only feature scaling will give a bad effect on our model, let's try dropping some unnecessary features now.

Training and evaluating for important features.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier_m_knn = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier_m_knn.fit(X_train_m, y_train)
classifier_m_knn.score(X_test_m,y_test)

After feature engineering we see a great decrement in the accuracy **from 93.33% to 59%**.